This is an example of the basic interface to control and get information from the robot.
The basic robot interface basically just inherits ur_rtde intefaces.
There is a lot more in that interface than what is shown here, it can be found in the ur_rtde documentation:
https://sdurobotics.gitlab.io/ur_rtde/api/api.html

# Robot Controller
To create a robot controller instance, we need the robot's information first: it's IP address.
it is saved in the robot metadata:


In [1]:
from ur_lab.robot_inteface.robots_metadata import ur5e_1
print("Robot metadata: ", ur5e_1)

Robot metadata:  {'name': 'ur5e_1', 'ip': '192.168.0.10'}


Create instance of robot controller and move the robot to the home configuration:

In [2]:
from ur_lab.robot_inteface.robot_interface import RobotInterface

robot = RobotInterface(ur5e_1["ip"])
robot.move_home()

Commands in this tutorial, such as move_home and others are not considering collisions (no motion planning). If you apply them and there is a collision or the robot is in a singular configuration, the robot will stop and you will have to manually move it back to a safe configuration.
If the robot collided in the last cell, just move it manually to the home config before you continue.

# Joint Space

The robot configuration is a vector of the angles (in radians) of all it's joints. There are 6 joints that can be between [-2pi, 2pi].
However, sometimes it is recommended to limit to [-pi, pi], as some joints cant reach the full range when the
robot is on a table, and reducing the range reduces the configuration space when motion planning. Also, when some joints reach the edges, cables can get wrapped around the robot and make it stop.

get current robot configuration, which is the home configuration:


In [3]:
q = robot.getActualQ()
print("robot configuration at home: ", q)

robot configuration at home:  [4.816054570255801e-06, -1.5708204708495082, 1.2699757711231996e-05, -1.5708075962462367, -1.2699757711231996e-05, -4.593526021778871e-06]


move it in each joint separately, by 10 degrees, then move back home (all joints at once):

In [4]:
import time

new_q = q.copy()
for i in range(6):
    new_q[i] += 20 * 3.14 / 180
    robot.moveJ(new_q, speed=0.3, acceleration=0.3)
    time.sleep(0.5)

In [5]:
robot.moveJ(q, speed=1, acceleration=1)

True

Note the speed and acceleration, this is the speed of the joint in rad/s, and the acceleration in rad/s^2.
It is recommended to do your tests with low speeds (~0.3) and when everything works safely you can increase.

## Move Path

There is an option to move the robot through a path of configurations, this is useful for complex motions.
It can simply be done by passing a list of configurations to the move_path function.


In [6]:
import numpy as np

# make sure robot at home first:
robot.move_home()

q_path = [[-np.pi/10, -np.pi/2 -np.pi/8, np.pi/4, -np.pi/2, 0, 0],
          [np.pi/10, -np.pi/2  -np.pi/6, np.pi/4, -np.pi/2, np.pi/5, 0],
          [np.pi/8, -np.pi/2 , np.pi/4, -np.pi/2, np.pi/5, np.pi/5],
          q]

robot.move_path(q_path, speed=0.3, acceleration=0.3, blend_radius=0.0)

Note how the robot moves to each config in the path, then slows down to a stop before moving to the next one.
If we want that movement to be continuous, which mean the robot will not slow to a complete stop before moving to the next configuration on the path, but will blend the movement between the configurations, we can set the blend_radius to a value greater than 0. This will make the robot move smoothly between the configurations.

This should look smoother and be faster since the robot doesn't reach a complete stop between the configurations:

In [7]:
robot.move_path(q_path, speed=0.3, acceleration=0.3, blend_radius=0.1)

## Move Asynchronously

Movement commands can be asynchronous, which means that the command will return immediately and the robot will start moving in the background while you can run code concurrently. This can be doen by passing the asynchronous=True argument to movement functions.
If another command is sent to the robot while it is moving, the robot will stop the current movement and start the new one.

In [8]:
robot.move_path(q_path, speed=0.3, acceleration=0.3, blend_radius=0.05, asynchronous=True)
time.sleep(0.5)
print("this is being printed while the robot is moving")
print("joint velocities:", robot.getActualQd())
time.sleep(5)
print("stopping robot in the middle of the path")
robot.stopJ()

robot.move_home()

this is being printed while the robot is moving
joint velocities: [-0.05922751873731613, -0.07367908209562302, 0.14341740310192108, 1.3514332248406553e-35, 7.551032467745245e-05, -0.0]
stopping robot in the middle of the path


# Cartesian Space

The robot can also be moved in the cartesian space, by passing the desired pose of the end effector.
This is movement in the coordinate system where the base of the robot is at (0,0,0) and the z-axis is pointing upwards, y-axis is pointing towards the cable that comes out from the base, and x is according to the right hand law.

The pose is defined as a 6D vector, where the first 3 elements are the position of the end effector, and the last 3 elements are the orientation of the end effector in the form of a rotation vector. Make sure not to confuse this 6D vector with the 6D vector of the robot configuration.

In [12]:
robot.move_home()

pose = robot.getActualTCPPose()
print("robot pose at home: ", pose)

robot pose at home:  [-0.003016535407284132, -0.3705908603790207, 1.0808999662546177, -0.011354115187130856, -2.2160817134552393, 2.222845602445737]


In [9]:
robot.moveJ([0, -np.pi/2, -np.pi/2, -np.pi/2, np.pi/2, 0], speed=0.3, acceleration=0.3)
print("robot pose at new configuration: ", robot.getActualTCPPose())

robot pose at new configuration:  [0.49279591878891704, -0.13504862510088955, 0.3507515837631036, 2.2183856150733154, -2.219684665442245, 0.009637519642462485]


The moveL command can move the end effector of the robot to a target in the cartesian space described above:

In [10]:
pose = robot.getActualTCPPose()
position = pose[:3]
orientation = pose[3:]

target_position = position.copy()
target_position[1] -= 0.1
target_pose = target_position + orientation
robot.moveL(target_pose, speed=0.03, acceleration=0.1)

True

Note that speed and acceleration here are in different units. Speed is in m/s and acceleration is in m/s^2 since we are moving in the cartesian space.
Typically, the speed should be lower here.

There is also a helper function for cartesian space movement relative to current end effector position, with fixed orientation:

In [11]:
# up and down:
robot.moveL_relative([0, 0, 0.1], speed=0.1, acceleration=0.1)
robot.moveL_relative([0, 0, -0.1], speed=0.1, acceleration=0.1)
# along x:
robot.moveL_relative([0.1, 0, 0], speed=0.1, acceleration=0.1)
robot.moveL_relative([-0.1, 0, 0], speed=0.1, acceleration=0.1)

Note that cartesian space movement is not collision-aware, so make sure the robot is in a safe configuration before moving in the cartesian space.
It is not recommended to move the robot in the cartesian space without a good reason and for too large movement, as small movements in cartesian space can be translated to complex and large movements in the joint space, this can lead to singularities, collisions, and other issues.

There is a lot more to learn. You are advised to browse:
https://sdurobotics.gitlab.io/ur_rtde/api/api.html